In [1]:
!pip install kaggle

Uploading the Kaggle.json file

In [2]:
# configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Importing Twitter Sentiment Dataset

In [3]:
#Using API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

 85% 69.0M/80.9M [00:00<00:00, 176MB/s]
100% 80.9M/80.9M [00:00<00:00, 183MB/s]


In [4]:
#Extracting the compressed dataset

from zipfile import ZipFile

dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The data is extracted successfully')

The data is extracted successfully


Importing the Dependencies

In [5]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
#printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Procseeing

In [8]:
#loading the data from csv file to pandas dataframe
df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [9]:
#checking the no of rows and columns in the dataset
df.shape

(1599999, 6)

In [10]:
#checking the 1st five rows of the dataset
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
#changing the name of the columns and reading the data again

column_names = ['target', 'id', 'date', 'flag', 'user', 'text']

df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names = column_names ,encoding = 'ISO-8859-1')

In [12]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [13]:
df.shape

(1600000, 6)

In [14]:
#checking the missing values in the dataset
df.isnull().sum()


target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [15]:
#checking the distribution of target column
df['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

Converting the target value from 4(positive) to 1

In [16]:
df.replace({'target':{4:1}}, inplace = True)

In [17]:
df['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

0 --> Negative tweet
1 --> Positive tweet

**Stemming**(reducing a word to its root word)

In [18]:
stemmer = PorterStemmer()

In [19]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [20]:
df['stemmed_content'] = df['text'].apply(stemming)

In [21]:
df.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [22]:
print(df['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [23]:
print(df['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [24]:
#Creating the independent and dependent variable for the model
X = df['stemmed_content'].values
Y = df['target'].values

In [25]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [26]:
print(Y)

[0 0 0 ... 1 1 1]


In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.8, stratify = Y, random_state = 2)

In [28]:
print("X shape: {}".format(X.shape))
print("X-Train Shape: {}".format(X_train.shape))
print("X-Test Shape: {}".format(X_test.shape))
print("Y shape: {}".format(Y.shape))
print("Y-Train Shape: {}".format(Y_train.shape))
print("Y-Test Shape: {}".format(Y_test.shape))

X shape: (1600000,)
X-Train Shape: (320000,)
X-Test Shape: (1280000,)
Y shape: (1600000,)
Y-Train Shape: (320000,)
Y-Test Shape: (1280000,)


In [29]:
#Converting the textual data into numerical data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [30]:
print(X_train)

  (0, 48457)	0.1783487752665268
  (0, 43092)	0.386842350151501
  (0, 167850)	0.240111467402399
  (0, 14346)	0.39931315003032014
  (0, 91021)	0.12279231293768048
  (0, 64953)	0.2818926236771396
  (0, 3258)	0.268863609257796
  (0, 169337)	0.19699933564194627
  (0, 2267)	0.386842350151501
  (0, 102778)	0.29737947041409823
  (0, 97427)	0.34420440374144945
  (0, 149213)	0.14186208446047013
  (0, 69568)	0.13827271618536066
  (1, 139545)	0.29888555194176414
  (1, 171582)	0.2484903566305166
  (1, 58742)	0.23532123603321103
  (1, 22248)	0.4730975135632803
  (1, 111490)	0.35682891305058717
  (1, 111560)	0.368031863026555
  (1, 138990)	0.24788749337516608
  (1, 176380)	0.1987217801344402
  (1, 24464)	0.4538829666013753
  (2, 103158)	0.25289862472456276
  (2, 56932)	0.1349989426882956
  (2, 173386)	0.14393122231218733
  :	:
  (319997, 129705)	0.5577275171539521
  (319997, 18924)	0.5338276940914474
  (319997, 87368)	0.34981217879815113
  (319997, 34978)	0.30029306772079845
  (319997, 154802)	0.2988

In [31]:
print(X_test)

  (0, 176345)	0.31673609293759325
  (0, 129575)	0.33441300429694437
  (0, 102503)	0.48768515882289265
  (0, 33928)	0.5144388444034379
  (0, 4929)	0.34898850449664237
  (0, 2487)	0.4044367378832719
  (1, 165253)	0.3124214918581995
  (1, 129575)	0.20657819537232627
  (1, 125210)	0.20948004318610883
  (1, 108557)	0.3825896035187392
  (1, 102600)	0.2904331109518877
  (1, 92010)	0.1974158149336613
  (1, 65439)	0.2571496816551475
  (1, 65263)	0.2351833778956644
  (1, 63539)	0.3016449922375899
  (1, 62435)	0.21841244452534805
  (1, 61988)	0.24569058915749686
  (1, 55709)	0.438138420687336
  (1, 52767)	0.18351335897572552
  (2, 160188)	0.21656745725725254
  (2, 159248)	0.24782890623285825
  (2, 157808)	0.1995440258352993
  (2, 147581)	0.5134867769008576
  (2, 145914)	0.30058676471077816
  (2, 144083)	0.22167376527081448
  :	:
  (1279995, 101280)	0.39536998652619587
  (1279995, 99397)	0.220300222649306
  (1279995, 87421)	0.3662534378184293
  (1279995, 25090)	0.2829472557167034
  (1279996, 66289

Training the Machine Learning Model

In [32]:
model = LogisticRegression(max_iter = 1000)

In [33]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Model Evalutaion

Accuracy Score

In [34]:
#accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [35]:
print('Accuracy score of training data: {}'.format(training_data_accuracy))

Accuracy score of training data: 0.815715625


In [36]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [37]:
print('Accuracy score of testing data: {}'.format(testing_data_accuracy))

Accuracy score of testing data: 0.7711046875


It means our model is not overfitted neither underfitted

Model accuracy = 78%

Saving the trained model

In [38]:
import pickle

In [39]:

# Create a dictionary
data = {'vectorizer': vectorizer, 'model': model}

# Save the dictionary to a pickle file
filename = 'model_and_vectorizer.pkl'
with open(filename, 'wb') as f:
    pickle.dump(data, f)

print("TF-IDF vectorizer and model saved to model_and_vectorizer.pkl")


TF-IDF vectorizer and model saved to model_and_vectorizer.pkl


In [40]:
# Create a dictionary
data = {'vectorizer': vectorizer, 'model': model}

# Save the dictionary to a pickle file
filename = 'model_and_vectorizer.sav'
with open(filename, 'wb') as f:
    pickle.dump(data, f)

print("TF-IDF vectorizer and model saved to model_and_vectorizer.sav")

TF-IDF vectorizer and model saved to model_and_vectorizer.sav


Using the created model for future prediction

In [41]:
test_new = "Elena is getting late every day, I think we should ban her"

In [42]:
filename = 'model_and_vectorizer.pkl'  # Or 'text_classifier.pkl' depending on the method

# Load the data
with open(filename, 'rb') as f:
    data = pickle.load(f)

In [43]:
loaded_vectorizer = data['vectorizer']

In [44]:
loaded_model = data['model']

In [45]:
processed_input = stemming(test_new)
print(processed_input)

elena get late everi day think ban


In [46]:
X_new = np.array([processed_input]).reshape(1, -1)

In [47]:
print(X_new)

[['elena get late everi day think ban']]


In [49]:
processed_text = ' '.join(X_new[0])  # Assuming X_new is a single-row array
X_new_transformed = loaded_vectorizer.transform([processed_text])

In [50]:
print(X_new_transformed)

  (0, 157808)	0.2443162995996281
  (0, 88518)	0.32759330113989865
  (0, 56932)	0.20305065552322768
  (0, 48477)	0.3527453665516711
  (0, 45437)	0.6129428915426676
  (0, 36622)	0.2049615291376428
  (0, 12569)	0.4996249550883103


In [51]:
prediction = loaded_model.predict(X_new_transformed)

In [52]:
if prediction[0] == 0:
    print("Sentiment: Negative")
else:
    print("Sentiment: Positive")

Sentiment: Negative
